In [1]:
import tensorflow.lite as tflite
from PIL import Image
import numpy as np

from io import BytesIO
from urllib import request

In [2]:
def preprocess_input(x):
    x /= 127.5
    x -= 1
    return x

In [3]:
def preprocess_img(img):
    x = np.array(img.resize((299, 299)), dtype='float32')
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X

In [4]:
def read_img_from_url(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    
    return preprocess_img(img)

In [5]:
def read_img_from_file(file_name):
    # Read the input image and resize
    with Image.open(file_name) as img:
        return preprocess_img(img)

In [6]:
X = read_img_from_url('http://bit.ly/mlbookcamp-pants')

In [7]:
# to include rescaling layer for xception
# To rescale an input in the [0, 255] range to be in the [-1, 1]

# layers.Rescaling(scale=1./127.5, offset=-1)

In [8]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
# interpreter.allocate_tensors()

In [9]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['input_8'], 'outputs': ['dense_7']}}

In [10]:
predictions_lite = interpreter.get_signature_runner('serving_default')(input_8=X)['dense_7']

In [11]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [12]:
dict(zip(classes, predictions_lite[0]))


{'dress': -1.9791993,
 'hat': -4.723823,
 'longsleeve': -2.1309288,
 'outwear': -1.049135,
 'pants': 9.9577465,
 'shirt': -3.027922,
 'shoes': -3.8510282,
 'shorts': 3.5252106,
 'skirt': -2.74625,
 't-shirt': -4.7122455}